### Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2
import timm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.core.transforms_interface import ImageOnlyTransform
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':128,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### Data Load

In [ ]:
all_img_list = glob.glob('./train/*/*')

In [ ]:
df = pd.DataFrame(columns=['img_path', 'rock_type'])
df['img_path'] = all_img_list
df['rock_type'] = df['img_path'].apply(lambda x: os.path.normpath(x).split(os.sep)[-2])


In [ ]:
df

In [ ]:
train, val, _, _ = train_test_split(df, df['rock_type'], test_size=0.1, stratify=df['rock_type'], random_state=CFG['SEED'])

In [ ]:
le = preprocessing.LabelEncoder()
train['rock_type'] = le.fit_transform(train['rock_type'])
val['rock_type'] = le.transform(val['rock_type'])

### Data Preprocessing

In [ ]:
class PadSquare(ImageOnlyTransform):
    def __init__(self, border_mode=0, value=0, always_apply=False, p=1.0):
        super().__init__(always_apply, p)
        self.border_mode = border_mode
        self.value = value

    def apply(self, image, **params):
        h, w, c = image.shape
        max_dim = max(h, w)
        pad_h = max_dim - h
        pad_w = max_dim - w
        top = pad_h // 2
        bottom = pad_h - top
        left = pad_w // 2
        right = pad_w - left
        image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=self.value)
        return image

    def get_transform_init_args_names(self):
        return ("border_mode", "value")

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transform = A.Compose([
    PadSquare(value=(0, 0, 0)),
    A.RandomResizedCrop(size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']), scale=(0.3, 1.0), ratio=(0.9, 1.1), p=1.0),
    A.Affine(scale=(0.9, 1.1), translate_percent=(0.05, 0.05), shear=(-5, 5), p=0.5),
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(limit=0.15, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.GridDropout(ratio=0.2, unit_size_min=5, p=0.5),
    ToTensorV2()
])

test_transform = A.Compose([
    PadSquare(value=(0, 0, 0)),
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [ ]:
from collections import Counter
class_counts = dict(Counter(train['rock_type']))
print("Class counts:", class_counts)

weights = [1.0 / class_counts[label] for label in train['rock_type']]

sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train['rock_type'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], sampler=sampler, num_workers=8,pin_memory=True,prefetch_factor=2)

val_dataset = CustomDataset(val['img_path'].values, val['rock_type'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=8,pin_memory=True,prefetch_factor=2)

### Train

In [ ]:
import numpy as np
import torch

class EarlyStopping:
    def __init__(self, patience=5, verbose=True, path="best_model_earlystop.pth"):
        self.patience = patience
        self.verbose = verbose
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, f1_score, model):

        score = f1_score

        if self.best_score is None:
            self.best_score = score
            self._save_checkpoint(model)
        elif score <= self.best_score:
            self.counter += 1
            if self.verbose:
                print(f"🔁 EarlyStopping counter: {self.counter} / {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self._save_checkpoint(model)
            self.counter = 0

    def _save_checkpoint(self, model):
        if self.verbose:
            print(f"F1 score improved. Saving model to {self.path}")
        torch.save(model.state_dict(), self.path)


In [ ]:
from torch.cuda.amp import GradScaler, autocast
from copy import deepcopy
from tqdm import tqdm
import os

def train(model, optimizer, train_loader, val_loader, scheduler, device, early_stopping):
    model.to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.05).to(device)
    scaler = GradScaler()

    best_score = 0
    best_model = None
    save_path = os.path.join(".", "best_model.pth")

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = []

        loop = tqdm(train_loader, desc=f"Epoch {epoch}", leave=True)
        for imgs, labels in loop:
            imgs = imgs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            with autocast():
                outputs = model(imgs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss.append(loss.item())
            loop.set_postfix(loss=loss.item())

        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)

        print(f'Epoch [{epoch}], Train Loss: {_train_loss:.5f}, Val Loss: {_val_loss:.5f}, Val Macro F1: {_val_score:.5f}', flush=True)

        if scheduler is not None:
            scheduler.step(_val_score)

        if best_score < _val_score:
            best_score = _val_score
            best_model = deepcopy(model)
            torch.save(model.state_dict(), save_path)
            print(f"Best model saved (epoch {epoch}, F1={_val_score:.4f}) → {save_path}", flush=True)

        early_stopping(_val_score, model)
        if early_stopping.early_stop:
            print("Early stopping triggered.", flush=True)
            break

    return best_model


In [ ]:
from torch.cuda.amp import autocast
from sklearn.metrics import f1_score

def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.to(device)
            labels = labels.to(device)

            with autocast():
                pred = model(imgs)
                loss = criterion(pred, labels)

            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            val_loss.append(loss.item())

    _val_loss = np.mean(val_loss)
    _val_score = f1_score(true_labels, preds, average='macro')

    return _val_loss, _val_score


In [ ]:
early_stopping = EarlyStopping(patience=5, verbose=True, path="tiny_vit384.pth")

model = timm.create_model(
    "hf_hub:timm/tiny_vit_21m_384.dist_in22k_ft_in1k",
    pretrained=True
)
model.reset_classifier(num_classes=7)


optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"], weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device, early_stopping)

In [ ]:
early_stopping = EarlyStopping(patience=5, verbose=True, path="effi_s.pth")

model = timm.create_model(
    "hf_hub:timm/tf_efficientnetv2_s.in21k_ft_in1k",
    pretrained=True
)
model.reset_classifier(num_classes=7)


optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"], weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device, early_stopping)

In [ ]:
early_stopping = EarlyStopping(patience=5, verbose=True, path="regnety_120.pth")

model = timm.create_model(
    "hf_hub:timm/regnety_120.sw_in12k_ft_in1k",
    pretrained=True
)
model.reset_classifier(num_classes=7)


optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"], weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device, early_stopping)

In [ ]:
early_stopping = EarlyStopping(patience=5, verbose=True, path="effi_m.pth")

model = timm.create_model(
    "hf_hub:timm/tf_efficientnetv2_m.in21k_ft_in1k",
    pretrained=True
)
model.reset_classifier(num_classes=7)


optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"], weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device, early_stopping)

### Inference

In [ ]:
model1 = timm.create_model(
    "hf_hub:timm/tiny_vit_21m_384.dist_in22k_ft_in1k",
    pretrained=False
)
model1.reset_classifier(num_classes=7)
model1.to(device)
model1.load_state_dict(torch.load('./tiny_vit384.pth', map_location=device))

model2 = timm.create_model(
    "hf_hub:timm/tf_efficientnetv2_s.in21k_ft_in1k",
    pretrained=False
)
model2.reset_classifier(num_classes=7)
model2.to(device)
model2.load_state_dict(torch.load('./effi_s.pth', map_location=device))

model3 = timm.create_model(
    "hf_hub:timm/regnety_120.sw_in12k_ft_in1k",
    pretrained=False
)
model3.reset_classifier(num_classes=7)
model3.to(device)
model3.load_state_dict(torch.load('./regnety_120.pth', map_location=device))

model4 = timm.create_model(
    "hf_hub:timm/tf_efficientnetv2_m.in21k_ft_in1k",
    pretrained=False
)
model4.reset_classifier(num_classes=7)
model4.to(device)
model4.load_state_dict(torch.load('./effi_m.pth', map_location=device))

In [ ]:
def apply_tta(imgs):
    tta_list = [
        imgs,
        torch.flip(imgs, dims=[3]),
        torch.flip(imgs, dims=[2]),
        torch.rot90(imgs, k=1, dims=[2, 3]),
        torch.rot90(imgs, k=2, dims=[2, 3]),
        torch.rot90(imgs, k=3, dims=[2, 3]),
        torch.transpose(imgs, 2, 3),
    ]
    return torch.cat(tta_list, dim=0)

def get_tta_probs(model, loader, device):
    model.eval()
    all_probs = []

    with torch.no_grad():
        for imgs in tqdm(loader, desc="Inference with TTA"):
            imgs = imgs.to(device)
            imgs_tta = apply_tta(imgs)

            with torch.cuda.amp.autocast():
                outputs = model(imgs_tta)
                probs = F.softmax(outputs, dim=1)

            
            B = imgs.shape[0]
            avg_probs = probs.view(7, B, -1).mean(dim=0)
            all_probs.append(avg_probs.cpu().numpy())

    return np.concatenate(all_probs, axis=0)

In [ ]:
test = pd.read_csv('./test.csv')
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
probs1 = get_tta_probs(model1, test_loader, device)
probs2 = get_tta_probs(model2, test_loader, device)
probs3 = get_tta_probs(model3, test_loader, device)
probs4 = get_tta_probs(model4, test_loader, device)


weighted_probs = probs1 + probs2 + probs3 + probs4
final_preds = np.argmax(weighted_probs, axis=1)
final_labels = le.inverse_transform(final_preds)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['rock_type'] = final_labels
submit.to_csv('./dacon_final.csv', index=False)